In [1]:
# Load the gams extension
%load_ext gams_magic

# Advertising Budget for Weasley's Wizard Wheezes

The Weasley's are buying advertising time on the radio in order to
reach wizards and witches.  There are a set $N$ of adds that they
could possibly purchase.  Each minute of advertising of type $j \in
N$ costs $c_j$.  If they purchase $x_j$ minutes of advertising of an
add of type $j$, then they will ensure that $\alpha_j \sqrt{x_j}$
witches and $\beta_j \sqrt{x_j}$ wizards hear their spots.

They would like to find a least cost advertising strategy so that
their ads will reach $K_1$ witches and $K_2$ wizards.

For both problems, please use the following gams code to create the instance.

In [2]:
%%gams
sets N /ad1*ad20/;
alias(I,N) ;

option seed = 1253;

parameters 
  c(I) Cost
  alpha(I) Witches proportionality constant
  beta(I) Wizards proportionality constant;

scalars K1, K2, totalAdTime;

c(I) = normal(100,5);

alpha(I) = uniform(7,13) ;
beta(I) = 13-alpha(I) + 7 + 5$(uniform(0,1) < 0.3);

K1 = 5000;
K2 = 8000;

### Write a GAMS nlp model that will determine the number of minutes of each advertising spot to purchase. 

Ensure your code works with the nlp solvers knitro, conopt and ipopt

In [3]:
%%gams
positive variable x(I);
free variable totcosts;

equations k1_limit "k1 limit"
          k2_limit "k2 limit"
          objfunc "objective function";
* Equation definitions
k1_limit..
  sum(I,alpha(I)*sqrt(x(I))) =g= K1;
  
k2_limit..
  sum(I,beta(I)*sqrt(x(I))) =g= K2;
    
* Objective function    
objfunc..
  totcosts =e= sum(I,c(I)*x(I));
  
model advert /all/;

* use different solvers to solve
option nlp = knitro;
solve advert using nlp min totcosts;

option nlp = conopt;
solve advert using nlp min totcosts;

option nlp = ipopt;
solve advert using nlp min totcosts;

totalAdTime = sum(I,x.l(I));

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),1.970912e+06,3,21,NLP,KNITRO,0.009
1,Normal (1),OptimalLocal (2),1.970912e+06,3,21,NLP,CONOPT,0.001
2,Normal (1),OptimalLocal (2),1.970912e+06,3,21,NLP,IPOPT,0.003


In [4]:
# Display the total time purchased and the 
# amount of each ad purchased using the code below:
%gams_pull x totalAdTime
totalAdTime = totalAdTime[0]
c = [(x[i][0],x[i][1]) for i in range(0,len(x))]
print("totalAdTime = ",totalAdTime)
display(c)

totalAdTime =  20139.5674256483


[('ad1', 1879.1193727324462),
 ('ad2', 300.7224667513591),
 ('ad3', 434.25035514573847),
 ('ad4', 1191.33313037181),
 ('ad5', 1555.6128652379837),
 ('ad6', 1175.9325191240034),
 ('ad7', 1067.6906060467222),
 ('ad8', 953.2716519426702),
 ('ad9', 333.74379722705476),
 ('ad10', 343.8519293866643),
 ('ad11', 529.4673136079932),
 ('ad12', 477.79775002878955),
 ('ad13', 2171.7042371726434),
 ('ad14', 2267.525660483433),
 ('ad15', 951.001883059992),
 ('ad16', 615.4953968309145),
 ('ad17', 375.7340854211762),
 ('ad18', 914.8816975429856),
 ('ad19', 1398.1083233667375),
 ('ad20', 1202.3223841671786)]

### Write a GAMS qcp model that will determine the number of minutes of each advertising spot to purchase.  
You will need to add some auxiliary variables.  
Ensure your code works with at least two of the solvers (cplexd, mosek and gurobi).

In [5]:
%%gams

* define a new positive variable u(I)
positive variable u(I);

free variable totcosts1;

equations k1_limit1
          k2_limit1
          constr_3(I) "add a new constraint on the new variable we created"
          objfunc1;
* Equation definitions
k1_limit1..
  sum(I,alpha(I)*u(I)) =g= K1;
  
k2_limit1..
  sum(I,beta(I)*u(I)) =g= K2;
    
constr_3(I)..
  x(I) =g= u(I) * u(I);
    
* Objective function
objfunc1..
  totcosts1 =e= sum(I,c(I)*x(I));
  
model advert1 /k1_limit1,k2_limit1,constr_3,objfunc1/;

* use different solvers to solve
option qcp = cplexd;
solve advert1 using qcp minimizing totcosts1;

option qcp = mosek;
solve advert1 using qcp minimizing totcosts1;

option qcp = gurobi;
solve advert1 using qcp minimizing totcosts1;

totalAdTime = sum(I,x.l(I));

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1.970907e+06,23,41,QCP,CPLEXD,0.025
1,Normal (1),Optimal Global (1),1.970912e+06,23,41,QCP,MOSEK,0.007
2,Normal (1),Optimal Global (1),1.970912e+06,23,41,QCP,GUROBI,0.018


In [6]:
# Display the total time purchased and the 
# amount of each ad purchased using the code below:
%gams_pull x totalAdTime
totalAdTime = totalAdTime[0]
c = [(x[i][0],x[i][1]) for i in range(0,len(x))]
print("totalAdTime = ",totalAdTime)
display(c)

totalAdTime =  20139.55932835093


[('ad1', 1879.1777059774681),
 ('ad2', 300.74399980862614),
 ('ad3', 434.24324620970395),
 ('ad4', 1191.3286567098933),
 ('ad5', 1555.6008643630857),
 ('ad6', 1175.941696945053),
 ('ad7', 1067.7243558473806),
 ('ad8', 953.2877755024334),
 ('ad9', 333.73433376572194),
 ('ad10', 343.84479425049784),
 ('ad11', 529.4528431128722),
 ('ad12', 477.79098009923246),
 ('ad13', 2171.7142760390393),
 ('ad14', 2267.485381177699),
 ('ad15', 951.010610352154),
 ('ad16', 615.4869948663686),
 ('ad17', 375.7298265452652),
 ('ad18', 914.8503335802889),
 ('ad19', 1398.0875333635586),
 ('ad20', 1202.323119834585)]